In [1]:
import numpy as np
import pandas as pd
import alpha_vantage
import time
from alpha_vantage.timeseries import TimeSeries 
from alpha_vantage.techindicators import TechIndicators
from config import config
from alpha_vantage.cryptocurrencies import CryptoCurrencies

In [8]:
#5 calls per minute, 500 per day
ts = TimeSeries(key=config["alpha_vantage"]["key"], output_format = "pandas")
ti = TechIndicators(key=config["alpha_vantage"]["key"], output_format='pandas')

stock_dataframes = {}

for symbol in config["alpha_vantage"]["symbol"]:
    print(symbol)
    time.sleep(60)
    ts_data, ts_meta_data = ts.get_daily_adjusted(symbol, outputsize=config["alpha_vantage"]["outputsize"])
    
    ema_data, ema_meta_data = ti.get_ema(symbol, interval="daily")
    rsi_data, rsi_meta_data = ti.get_rsi(symbol, interval="daily")
    time.sleep(60)
    macd_data, macd_meta_data = ti.get_macd(symbol, interval="daily")
    bband_data, bband_meta_data = ti.get_bbands(symbol, interval="daily")
    ad_data, ad_meta_data = ti.get_ad(symbol, interval="daily")
    
    stock_dataframes[symbol] = pd.concat([ts_data, ema_data, rsi_data, macd_data, bband_data, ad_data], axis=1, join="inner")
    stock_dataframes[symbol].drop(['1. open', '2. high', '3. low', '5. adjusted close', '7. dividend amount', '8. split coefficient'], axis=1, inplace=True)
    stock_dataframes[symbol]

IBM
GOOG
NFLX
AMZN
AAPL
MSFT
TSLA
NVDA
ADBE
INTC


In [12]:
for symbol in stock_dataframes:
    stock_dataframes[symbol].to_csv('StockData/' + symbol + '.csv', index=True)

In [16]:
normalized_stock_dataframes = {}
for symbol in stock_dataframes:
    normalized_stock_dataframes[symbol] = (stock_dataframes[symbol]-stock_dataframes[symbol].min())/(stock_dataframes[symbol].max()-stock_dataframes[symbol].min())
    normalized_stock_dataframes[symbol].to_csv('NormalizedStockData/' + symbol + '.csv', index=True)
    


In [31]:
train_data = {}
test_data = {}
for symbol in normalized_stock_dataframes:
    train_size = int(len(normalized_stock_dataframes[symbol]) * config['data']['train_split_size'])
    test_size = len(normalized_stock_dataframes[symbol]) - train_size
    train_data[symbol] = normalized_stock_dataframes[symbol][test_size:-1]
    test_data[symbol] = normalized_stock_dataframes[symbol][0:test_size]


,4. close,6. volume,EMA,RSI,MACD,MACD_Hist,MACD_Signal,Real Upper Band,Real Lower Band,Real Middle Band,Chaikin A/D
date,,,,,,,,,,,
2017-07-10,0.611896,0.063683,0.768807,0.498473,0.678145,0.537172,0.682847,0.715877,0.783869,0.766190,0.937974
2017-07-07,0.608909,0.052491,0.769191,0.476908,0.679287,0.536700,0.684435,0.715790,0.784421,0.766431,0.937141
2017-07-06,0.605301,0.061999,0.769980,0.450614,0.683461,0.543721,0.686087,0.716712,0.782924,0.766133,0.936633
2017-07-05,0.613451,0.082064,0.771290,0.496822,0.691962,0.563349,0.686897,0.718716,0.779976,0.765644,0.939312
2017-07-03,0.625334,0.064115,0.771746,0.573940,0.695873,0.576362,0.685373,0.719211,0.778590,0.765182,0.943327
...,...,...,...,...,...,...,...,...,...,...,...
1999-12-27,0.340198,0.086503,0.243484,0.654166,0.803586,0.428128,0.884346,0.250950,0.259235,0.253625,0.408263
1999-12-23,0.333167,0.105715,0.242462,0.635441,0.810414,0.414677,0.898885,0.250982,0.256764,0.252362,0.405052
1999-12-22,0.329310,0.092961,0.241971,0.625220,0.821872,0.409478,0.915028,0.251276,0.254356,0.251266,0.408324


NameError: name 'IBM_data' is not defined